In [1]:
from pyspark.sql import SparkSession

# Creating Spark session
spark = SparkSession.builder \
                    .appName("CPT Codes Ingestion") \
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/23 09:24:30 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
25/08/23 09:24:30 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
25/08/23 09:24:30 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
25/08/23 09:24:30 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
# configuring variables
BUCKET_NAME = "healthcare-bucket-14082025"
CPT_BUCKET_PATH = f"gs://{BUCKET_NAME}/landing/cptcodes/*.csv"
BQ_TABLE = "infra-memento-469009-a5.bronze_dataset.cpt_codes"
TEMP_GCS_BUCKET = f"{BUCKET_NAME}/temp/"

In [3]:
# reading from cpt
cptcodes_df = spark.read.csv(CPT_BUCKET_PATH, header=True)

In [4]:
# replacing spaces with underscore
for col in cptcodes_df.columns:
    new_col = col.replace(" ", "_").lower()
    cptcodes_df = cptcodes_df.withColumnRenamed(col, new_col)

In [5]:
# writing to bigquery
(cptcodes_df.write
            .format("bigquery")
            .option("table", BQ_TABLE)
            .option("temporaryGcsBucket", TEMP_GCS_BUCKET)
            .mode("overwrite")
            .save())